In [1]:
import pandas as pd
from sklearn.decomposition import NMF
from utils.process import prepare_df, data_arrays, data_tensors, LABELS
from sklearn import metrics
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from ssnmf.ssnmf import SSNMF_T
from sklearn.model_selection import KFold, StratifiedKFold

In [2]:
df = prepare_df()
df = df[df['Label'] != 'BENIGN']
df = df.sample(1000)
df.groupby('Label')['Label'].count()

Label
Bot                           3
DDoS                        247
DoS GoldenEye                22
DoS Hulk                    416
DoS Slowhttptest             11
DoS slowloris                12
FTP-Patator                  12
PortScan                    260
SSH-Patator                  11
Web Attack   Brute Force      4
Web Attack   XSS              2
Name: Label, dtype: int64

In [5]:
from ssnmf import SSNMF
import random

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


def get_Y_torch(X, y):
    y_unique = torch.unique(y)
    sample_size = X.shape[1]
    Y = torch.zeros(y_unique.shape[0], sample_size, device=device)
    for i in range(sample_size):
        j = ((y_unique == y[i]).nonzero(as_tuple=True)[0])
        Y[j, i] = 1
    return Y


def get_L_torch(Y, fraction_known=0.5):
    L = torch.zeros(Y.shape, device=device)
    m, n = L.shape
    num_samples = int(fraction_known * n)
    print(num_samples)
    labeled_data = torch.randperm(n, dtype=torch.int32, device=device)[:num_samples]
    for i in labeled_data:
        L[:,i] = torch.tensor(1)
    return L

#model = SSNMF(M_s,10, modelNum=1)

In [10]:
import torch


X, y = data_tensors(df)
Y = get_Y_torch(X.T, y)
L = get_L_torch(Y, fraction_known=0.5)
snmf = SSNMF_T(X.T, 11, Y=Y, L=L, lam=1 * np.linalg.norm(X), modelNum=3, str='torch')
snmf.mult(numiters=1000)
y_pred = torch.argmax(snmf.S, axis=0)

499


/home/tosia/NASK/network_analyze_new/utils/process.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace([np.inf, -np.inf], np.nan, inplace=True)


In [11]:
metrics.rand_score(y, y_pred)

0.7958585174360757

In [6]:
Y

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [12]:
skf = StratifiedKFold(n_splits=5)
fraction_known = 0.1
for k in [5, 10, 20, 30]:
    for lam in (0, 0.001, 0.5, 0.1, 0.5, 1, 10, 100, 500, 1000, 5000):
        for train_index, test_index in skf.split(X, y):
            X_train_T, X_test_T = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            Y_train = get_Y_torch(X_train_T.T, y_train)
            L_train = get_L_torch(Y_train, fraction_known=fraction_known)
            #print(f"norm X {np.linalg.norm(X_train_T)}")
            snmf = SSNMF(X_train_T.T, k, L=L_train, Y=Y_train, lam=lam * np.linalg.norm(X_train_T), modelNum=5)
            snmf.mult(numiters=1000)
            y_pred = np.argmax(snmf.S, axis=0)
            score = metrics.rand_score(y_train, y_pred)
            print(f'lambda {lam} score {score}')

/home/tosia/.pyenv/versions/3.9.5/lib/python3.8/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


lambda 0 score 0.7323717146433041
lambda 0 score 0.8102503128911139
lambda 0 score 0.8070025031289111
lambda 0 score 0.7240894868585732
lambda 0 score 0.8081289111389236
lambda 0.001 score 0.7485419274092616
lambda 0.001 score 0.7476752190237798
lambda 0.001 score 0.7254036295369212


/home/tosia/.pyenv/versions/3.9.5/lib/python3.8/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


lambda 0.001 score 0.7879192740926158
lambda 0.001 score 0.8094430538172716
lambda 0.5 score 0.9117021276595745


/home/tosia/.pyenv/versions/3.9.5/lib/python3.8/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


...

lambda 500 score 0.7850438047559449
lambda 500 score 0.8060012515644556
lambda 500 score 0.7727878598247809
lambda 500 score 0.7741864831038798
lambda 500 score 0.7825719649561953
lambda 1000 score 0.7824405506883605


/home/tosia/.pyenv/versions/3.9.5/lib/python3.8/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


lambda 1000 score 0.7892459324155194
lambda 1000 score 0.8099530663329162
lambda 1000 score 0.7778097622027534
lambda 1000 score 0.7871057571964957


/home/tosia/.pyenv/versions/3.9.5/lib/python3.8/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


lambda 5000 score 0.7698247809762203
lambda 5000 score 0.7758260325406758
lambda 5000 score 0.7714549436795995
lambda 5000 score 0.8030632040050063
lambda 5000 score 0.8086983729662077
